In [ ]:
"""
Created on Tue Jun 27 16:57 2023

Look at the grounding line flux from ABUMIP

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import glob
import numpy as np

import basal_melt_param.create_isf_mask_functions as isfm
import basal_melt_param.useful_functions as uf

In [ ]:
%matplotlib qt5

In [ ]:
inputpath = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/ABUMIP_DATA/'

In [ ]:
file_list = glob.glob(inputpath+'/*/ligroundf*.nc')

In [ ]:
file_for_coord = xr.open_dataset(inputpath + 'DOE_MALI_abuk/ligroundf_AIS_DOE_MALI_abuk.nc',decode_times=False)
xx = file_for_coord.x
yy = file_for_coord.y

In [ ]:
GL_list = []
for ff in file_list:
    mod_name_sep = ff.split('/')[7].split('_')[0:2]
    mod_name = '_'.join(mod_name_sep)
    print(mod_name)
    
    GL_flux_ff = xr.open_dataset(ff,decode_times=False)
    GL_flux_mod = GL_flux_ff['ligroundf'].assign_coords({'model': mod_name, 'time': np.arange(len(GL_flux_ff.time)), 'y': yy, 'x': xx})
    GL_list.append(GL_flux_mod)
    if mod_name not in ['CPOM_BISICLES', 'NCAR_CISM']:
        print(GL_flux_mod.units)

GL_ds = xr.concat(GL_list, dim='model')

In [ ]:
for mmod in GL_ds.model:
    plt.figure()
    GL_ds.mean('time').sel(model=mmod).plot()

In [ ]:
for mmod in GL_ds.model:
    if mmod not in ['IMAU_IMAUICE','NCAR_CISM','CPOM_BISICLES']:
        GL_ds.sel(model=mmod).sum(['x','y']).plot(label=mmod.values)
plt.legend()

In [ ]:
outputpath_mask='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_OPM016/'
whole_ds = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = whole_ds['Nisf'].where(np.isfinite(whole_ds['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = whole_ds.sel(Nisf=nonnan_Nisf)
#large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 3200, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
isf_mask = file_isf['ISF_mask']

In [ ]:
GL_flux_list = []
for kisf in file_isf.Nisf:
    GL_flux_kisf = GL_ds.where(isf_mask == kisf).sum(['x','y'])
    GL_flux_list.append(GL_flux_kisf.assign_coords({'Nisf':kisf}))
GL_flux_timeseries = xr.concat(GL_flux_list, dim='Nisf')

In [ ]:
plt.close('all')

In [ ]:
for kisf in GL_flux_timeseries.Nisf:
    plt.figure()
    for mmod in GL_flux_timeseries.model:
        GL_flux_timeseries.sel(model=mmod, Nisf=kisf).plot(label=mmod.values)
    plt.legend()
    plt.title(file_isf['isf_name'].sel(Nisf=kisf).values)

In [ ]:
kisf = 66
plt.figure()
for mmod in GL_flux_timeseries.model:
    GL_flux_timeseries.sel(model=mmod, Nisf=kisf).plot(label=mmod.values)
plt.legend()
plt.title(file_isf['isf_name'].sel(Nisf=kisf).values)
#plt.ylim(-5,5)